In [2]:
import pandas as pd

# Load your metadata
metadata = pd.read_csv("../data/Data/manifest/all_data.csv")

# Drop duplicates by patient_id + laterality + view
metadata = metadata.drop_duplicates(
    subset=["patient_id", "left or right breast", "image view"],
    keep="first"
)

metadata.to_csv("../data/Data/manifest/all_data_processed.csv", index=False)
